In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from glob import glob
import json
import os
import sys
sys.path.append(os.path.abspath(os.getcwd() + "./../"))
print(sys.path)

# load test dataset
from torch.utils.data import DataLoader
import json

import json
from tqdm import tqdm
import numpy as np
import os
import argparse

import torch
from torch import nn, optim
from torchvision import transforms as T
import json

from tasks.seq_mnist import SeqMNISTModelTraining, SeqMNISTParams, SequentialMNIST

%matplotlib inline

['/Users/kerekmen/miniconda3/envs/s4/lib/python312.zip', '/Users/kerekmen/miniconda3/envs/s4/lib/python3.12', '/Users/kerekmen/miniconda3/envs/s4/lib/python3.12/lib-dynload', '', '/Users/kerekmen/miniconda3/envs/s4/lib/python3.12/site-packages', '/Users/kerekmen/Studium/Master/Semester2/NN/nn-seminar', '/Users/kerekmen/Studium/Master/Semester2/NN/nn-seminar', '/Users/kerekmen/Studium/Master/Semester2/NN/nn-seminar', '/Users/kerekmen/Studium/Master/Semester2/NN/nn-seminar', '/Users/kerekmen/Studium/Master/Semester2/NN/nn-seminar', '/Users/kerekmen/Studium/Master/Semester2/NN/nn-seminar']


In [14]:
# load test dataset
task_params = {
    "resize_resolution": 8,
}
test_dataset = SequentialMNIST(task_params, train=True)

# load the model
# model = SeqMNISTModelTraining()
# model.net.load_state_dict(torch.load('/Users/kerekmen/Desktop/pytorch-ntm-master/checkpoints/seq_mnist-task-1000-batch-930-2024-06-0823:35:16.318871.model'))

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:04<00:00, 2020048.31it/s]


Extracting ./data/sequential_mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/sequential_mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 244748.24it/s]


Extracting ./data/sequential_mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/sequential_mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1612826.74it/s]


Extracting ./data/sequential_mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/sequential_mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 2902274.34it/s]

Extracting ./data/sequential_mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/sequential_mnist/MNIST/raw



# Run eval

In [15]:
print(f"Number of samples: {len(test_dataset)}")
model.net.eval()
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# test the model
correct = 0
total = 0
for i, (X, Y) in enumerate(test_loader):
    X = X.permute(1, 0, 2)
    Y = Y.squeeze(1)
    print(X.size(), Y.size())
    
    batch_size = X.size(1)
    model.net.init_sequence(batch_size)

    # Forward pass
    outputs = model.net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += targets.size(0)
    correct += (predicted == targets).sum().item()
print(f"Accuracy: {100 * correct / total:.2f}%")

Number of samples: 10000
torch.Size([64, 256, 1]) torch.Size([256])


RuntimeError: Tensors must have same number of dimensions: got 3 and 2